# Add Folders to Path

In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add EllipticOECS folder to current working path in order to access the functions
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/FastTensorlineComputation")

CPU times: user 141 µs, sys: 109 µs, total: 250 µs
Wall time: 175 µs


In [2]:
# Suppress numba warnings arising from computation falling back into object mode.
from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)

# Import data

In [3]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../../data/ABC/ABC_steady.mat')

U = mat_file['u']
V = mat_file['v']
W = mat_file['w']
x = mat_file['x']
y = mat_file['y']
z = mat_file['z']

CPU times: user 59.6 ms, sys: 69.8 ms, total: 129 ms
Wall time: 128 ms


# Data/Parameters for Dynamical System

In [4]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 18

# periodic boundary conditions
periodic_x = True
periodic_y = True
periodic_z = True
periodic = [periodic_x, periodic_y, periodic_z]

# unsteady velocity field
bool_unsteady = False

## compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z)

## resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0]
dy_data = Y[1,0,0]-Y[0,0,0]
dz_data = Z[0,0,1]-Z[0,0,0]

delta = [dx_data, dy_data, dz_data]

# Spatio-Temporal Domain of Dynamical System

In [5]:
%%time
from math import pi

# Time
t0 = 0
tN = 10
dt = 0.1

# store time in array
time = np.arange(t0, tN+dt, dt)

lenT = 10

# domain boundary (in degrees)
xmin = 0
xmax = 2*pi
ymin = 0
ymax = 2*pi
zmin = 0
zmax = 2*pi

# spacing of meshgrid (in degrees)
dx = 0.1
dy = 0.1
dz = 0.1

x_domain = np.linspace(xmin, xmax, int((xmax-xmin)/dx), endpoint = True)
y_domain = np.linspace(ymin, ymax, int((ymax-ymin)/dy), endpoint = True)
z_domain = np.linspace(ymin, ymax, int((zmax-zmin)/dz), endpoint = True)

X_domain, Y_domain, Z_domain = np.meshgrid(x_domain, y_domain, z_domain)

CPU times: user 1.73 ms, sys: 1.85 ms, total: 3.58 ms
Wall time: 2.47 ms


# Interpolate Velocity

In order to evaluate the velocity field at arbitrary locations and times, we must interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". In order to favour a smooth velocity field, we interpolate the velocity field in space using a cubic interpolant. 

In [6]:
%%time
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_steady

# Interpolate velocity data using linear spatial interpolation
Interpolant = interpolant_steady(X, Y, Z, U, V, W)

CPU times: user 36.9 ms, sys: 15.9 ms, total: 52.8 ms
Wall time: 52.1 ms


# Rate of Strain $ \mathrm{S}(\mathbf{x})$

In [7]:
%%time
# Import package to compute gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# Import package to compute rate of strain
from ipynb.fs.defs.RateStrain import RateStrain

# Import package for progress bar
from tqdm.notebook import tqdm

# Import package for parallel computing
from joblib import Parallel, delayed

# Define auxiliary grid 
aux_grid = [np.around(.1*dx, 5), np.around(.1*dy, 5), np.around(.1*dz, 5)]

def parallel_S(i):
    
    S_parallel = np.zeros((X_domain.shape[1], X_domain.shape[2], 3, 3))*np.nan
    
    for j in range(X_domain.shape[1]):
        
        for k in range(Y_domain.shape[2]):
        
            # set initial condition
            x = np.array([X_domain[i, j, k], Y_domain[i, j, k], Z_domain[i, j, k]])
    
            # compute gradient of velocity
            grad_vel = gradient_velocity(time, x, X, Y, Z, Interpolant, periodic, bool_unsteady, aux_grid)
        
            # compute rate of strain
            S_parallel[j,k,:,:] = RateStrain(grad_vel)
        
    return S_parallel

S = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_S)(i) for i in tqdm(range(X_domain.shape[0]))))

  0%|          | 0/62 [00:00<?, ?it/s]

CPU times: user 456 ms, sys: 300 ms, total: 756 ms
Wall time: 1min 33s


# Compute Tensorfield properties

We now compute the properties of the rate of strain tensor 'S' such as the spatial derivatives of the elements of 'S'.

In [8]:
%%time
# Import (cubic) RectangularGridInterpolat from scipy
from scipy.interpolate import RegularGridInterpolator
        
# Compute gradients of elements of rate of strain tensor
S11 = np.nan_to_num(S[:,:,:,0,0], nan=0.0)
S12 = np.nan_to_num(S[:,:,:,0,1], nan=0.0)
S22 = np.nan_to_num(S[:,:,:,1,1], nan=0.0)
S33 = np.nan_to_num(S[:,:,:,2,2], nan=0.0)
S23 = np.nan_to_num(S[:,:,:,1,2], nan=0.0)
S13 = np.nan_to_num(S[:,:,:,0,2], nan=0.0)

# Interpolate elements of rate of strain tensor (per default: cubic)
interpS11 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S11)
interpS12 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S12)
interpS22 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S22)
interpS33 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S33)
interpS23 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S23)
interpS13 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S13)

CPU times: user 14.7 ms, sys: 3.63 ms, total: 18.3 ms
Wall time: 16.8 ms


In [10]:
# import math tools
from math import atan, acos, asin, cos, sin, sqrt

# Import package to compute tensorline equation using ODE solver
from ipynb.fs.defs.tensorline_equation import _tensorline_equation

# Import package to compute tensorlines by re-orienting the vectorfield on the fly
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines

# Import package to solve ODE
from scipy.integrate import solve_ivp

# Import package to solve ODE
from ipynb.fs.defs.eigen import eigen

# Import package for parallel computing
from joblib import Parallel, delayed

# import plotting library
import matplotlib.pyplot as plt

# define integration interval of dummy variable 's'
t = [0, 10]
t_eval = np.linspace(t[0], t[-1], 1000)

# initial alignment of eigenvector
init_orientation = np.array([0, 0, 1])

# define eigenvector_field 
# {0: eigenvector field of maximum eigenvalue}
# {1: eigenvector field of middle eigenvalue}
# {2: eigenvector field of minimum eigenvalue}
idx_eigenvector = 1

x_0 = X_domain[::10,::10,0].ravel()
y_0 = Y_domain[::10,::10,0].ravel()
z_0 = 0

def terminate_ODE(t, x_phi, interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, aux_grid, idx_eigenvector):
                
    x = x_phi[0]%(2*pi)
    y = x_phi[1]%(2*pi)
    z = x_phi[2]%(2*pi)
    
    # compute gradient of velocity
    grad_vel = gradient_velocity(0, np.array([x, y, z]), X, Y, Z, Interpolant, periodic, bool_unsteady, aux_grid)
    
    # compute rate of strain
    S = RateStrain(grad_vel)
    
    # compute eigenvalues/eigenvectors of rate of strain
    lam = eigen(S)[0]
            
    event = abs(lam[0]-lam[1]) <= .01 or abs(lam[0]-lam[2]) <= .01 or abs(lam[1]-lam[2]) <= .01
                
    return 1-event
            
terminate_ODE.terminal = True

def compute_e2(i):
            
    # initial position of particle
    xinit = np.array([x_0[i], y_0[i], z_0])
            
    # compute gradient of velocity
    grad_vel = gradient_velocity(t[0], xinit, X, Y, Z, Interpolant, periodic, bool_unsteady, aux_grid)
        
    # compute rate of strain
    S = RateStrain(grad_vel)
            
    # compute inital eigenvector orientation
    lamda, eigenv = eigen(S)
    initial_vector = eigenv[:,idx_eigenvector]
    print("initial particle position: ", xinit)
    print("initial vector: ", initial_vector)
    if np.sign(init_orientation@initial_vector) < 0:
        initial_vector = -initial_vector
            
    if initial_vector[0] < 0 or (initial_vector[0] < 0 and initial_vector[1] < 0):
        phi0 = atan(initial_vector[1]/initial_vector[0])+pi
    else:
        phi0 = atan(initial_vector[1]/initial_vector[0])
    theta0 = acos(initial_vector[2])
                
    x0 = np.array([x_0[i], y_0[i], z_0, phi0, theta0])
            
    # Compute tensorline by re-orienting vector field
    ds = t_eval[1]-t_eval[0]
    x_oriented = x0[:3].copy()
    x_prime = initial_vector.copy()
    trajectory_oriented = [[], [], []]
    trajectory_oriented[0].append(x_oriented[0])
    trajectory_oriented[1].append(x_oriented[1])
    trajectory_oriented[2].append(x_oriented[2])   
    phi_oriented, theta_oriented = [], []
    print("initial vector: ", x_prime)
            
    for idx_t in range(len(t_eval)):             
        x_oriented, x_prime = _RK4_tensorlines(X, Y, Z, x_oriented, x_prime, ds, periodic, interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, idx_eigenvector)
        trajectory_oriented[0].append(x_oriented[0])
        trajectory_oriented[1].append(x_oriented[1])
        trajectory_oriented[2].append(x_oriented[2])   
        vx = x_prime[0]
        vy = x_prime[1]
        vz = x_prime[2]
        norm_v = sqrt(vx**2+vy**2+vz**2)
        if norm_v == 0:
            phi_oriented.append(np.nan)
            theta_oriented.append(np.nan)                   
        else:
            if vx < 0:
                phi_oriented.append((atan(vy/vx)+pi)%(2*pi))
            else:
                phi_oriented.append((atan(vy/vx))%(2*pi))
            theta_oriented.append(acos(vz/norm_v))
            
    # solve ODE for tensorlines maximum eigenvector field of rate of strain
    solODE = solve_ivp(_tensorline_equation, t, x0, 'RK45', t_eval, rtol=1e-5, atol=1e-5, events = terminate_ODE, args=(interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, aux_grid, idx_eigenvector))
                               
    x = solODE.y[0,:]%(2*pi)
    y = solODE.y[1,:]%(2*pi)
    z = solODE.y[2,:]%(2*pi)
    phi = solODE.y[3,:]%(2*pi)
    theta = solODE.y[4,:]%pi
                
    # plot trajectory in 3D
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (14, 10), dpi = 500)
    ax1.scatter(trajectory_oriented[0][::5], trajectory_oriented[1][::5], facecolor = "None", edgecolor = "r", label = r'manual', marker = "o", s = 50)
    ax2.scatter(trajectory_oriented[0][::5], trajectory_oriented[2][::5], facecolor = "None", edgecolor = "r", label = r'manual', marker = "o", s = 50)
    ax3.scatter(trajectory_oriented[1][::5], trajectory_oriented[2][::5], facecolor = "None", edgecolor = "r", label = r'manual', marker = "o", s = 50)
    ax1.scatter(x[::5], y[::5], marker = "^", label = r'automatic', s = 50, facecolor = "None", edgecolor = "b")
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax1.set_title("projection onto x/y plane")
    ax2.set_xlabel("x")
    ax2.set_ylabel("z")
    ax2.scatter(x[::5], z[::5], marker = "^", label = r'automatic', s = 50, facecolor = "None", edgecolor = "b")
    ax2.set_title("projection onto x/z plane")
    ax3.scatter(y[::5], z[::5], marker = "^", label = r'automatic', s = 50, facecolor = "None", edgecolor = "b")
    ax3.set_xlabel("y")
    ax3.set_ylabel("z")
    ax3.set_title("projection onto y/z plane")
    ax4.scatter(phi, theta, label = r'automatic', marker = "^", s = 50, facecolor = "None", edgecolor = "b")
    ax4.scatter(phi_oriented, theta_oriented, label = r'manual', marker = "o", s = 50, facecolor = "None", edgecolor = "r")
    ax4.set_xlabel(r'$\phi$')
    ax4.set_ylabel(r'$\theta$')
    ax4.set_title(r'$ \theta - \phi $')
    ax1.scatter(trajectory_oriented[0][0], trajectory_oriented[1][0], c = "b", marker = "^", s = 100)
    ax2.scatter(trajectory_oriented[0][0], trajectory_oriented[2][0], c = "b", marker = "^", s = 100)
    ax3.scatter(trajectory_oriented[1][0], trajectory_oriented[2][0], c = "b", marker = "^", s = 100)
    plt.legend(loc="upper right", fontsize = 10)
            
    plt.savefig('./Fig/mid/fig_'+np.str(int(i)) + '.jpg')
    plt.close()
    
    return solODE

Ncores = 1
sol = Parallel(n_jobs=Ncores, verbose = 0)(delayed(compute_e2)(i) for i in tqdm(range(len(x_0))))


  0%|          | 0/49 [00:00<?, ?it/s]

initial particle position:  [0. 0. 0.]
initial vector:  [ 0.14178314 -0.80409216  0.57735027]
initial vector:  [ 0.14178314 -0.80409216  0.57735027]
initial particle position:  [1.03003038 0.         0.        ]
initial vector:  [-0.83319734  0.18087748  0.52255672]
initial vector:  [-0.83319734  0.18087748  0.52255672]
initial particle position:  [2.06006076 0.         0.        ]
initial vector:  [ 0.85201705 -0.20445654  0.48193823]
initial vector:  [ 0.85201705 -0.20445654  0.48193823]
initial particle position:  [3.09009113 0.         0.        ]
initial vector:  [-0.12109812  0.78947581  0.60171686]
initial vector:  [-0.12109812  0.78947581  0.60171686]
initial particle position:  [4.12012151 0.         0.        ]
initial vector:  [-0.28839693  0.934719    0.20767186]
initial vector:  [-0.28839693  0.934719    0.20767186]
initial particle position:  [5.15015189 0.         0.        ]
initial vector:  [ 0.29510222 -0.94296129  0.15407364]
initial vector:  [ 0.29510222 -0.94296129

initial particle position:  [6.18018227 6.18018227 0.        ]
initial vector:  [ 0.12860264 -0.8040732   0.58045469]
initial vector:  [ 0.12860264 -0.8040732   0.58045469]
